<p align="center">
    <!--If notebook is in folder notebooks change scr by "../images/logo.png"-->
  <img width="100" height="100" src="../../images/logo.png">
</p>

<div>
<h1>0X. Template</h1> 

Canadian Car Accidents Practice <br>
<strong>Aprendizaje Automático</strong> <br>
<strong>Master Universitario en Ciencia de Datos</strong>
</div>

<div style='text-align:right'>Álvaro Serrano del Rincón (<i>a.serranodelrincon@cunef.edu</i>)</div>
<div style='text-align:right'>Carlos Viñals Guitart (<i>carlos.vinals@cunef.edu</i>)</div>

---


## X.0 Introducción

< EXPLICACIÓN DEL CONTENIDO DEL NOTEBOOK>

Para este trabajo estamos utilizando un entorno de propósito espécifico.

In [4]:
!conda info


     active environment : ML_P1
    active env location : C:\Users\carviagu\anaconda3\envs\ML_P1
            shell level : 2
       user config file : C:\Users\carviagu\.condarc
 populated config files : 
          conda version : 4.10.3
    conda-build version : 3.21.4
         python version : 3.8.8.final.0
       virtual packages : __cuda=11.3=0
                          __win=0=0
                          __archspec=1=x86_64
       base environment : C:\Users\carviagu\anaconda3  (writable)
      conda av data dir : C:\Users\carviagu\anaconda3\etc\conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/msys2/win-64
                          https://repo.anaconda.com/pkgs/msys2/no

< AÑADIR SUBSECCIÓN CON ESTRUCTURA >

# 6.1. Introducción
La interpretabilidad es una parte fundamental de cualquier modelo de Machine Learning, de cara a poder poner en producción los modelos desarrollados. Existen fundamentalmente (y a muy grandes rasgos) dos clases de modelos de predicción, por lo menos en lo que a interpretabilidad se refiere:

* **Modelos lineales.** Tienen una interpretación más sencilla, pues los propios coeficientes indican tanto el peso como el signo, que indican cómo es el impacto de la variable. Además, también se realizan contrastes de significatividad, tanto a nivel individual como conjunto. 
* **Modelos no lineales.** Su interpretación es más compleja, pues requiere de unos cálculos adicionales a fin de determinar la importancia de cada variable en el modelo. 

Para la solución del caso planteado, se ha planteado un modelo **no lineal**, por lo que será necesario plantear una interpretación del mismo.

# 6.2. SHAP (SHapley Additive exPlanations)

El método seleccionado para explicar el modelo en cuestión es el SHAP (SHapley Additive exPlanations). Dicho método parte de **Shapley Values**, que es un modelo de redistribución de la riqueza, dentro de la **teoría de juegos cooperativos**. 

## 6.2.1. Shapley Values

No obstante, y antes de proceder a realizar los cálculos de interpretabilidad, sería necesario comprender la teoría de juegos sobre la que se basa el método. 

**Shapley Values** es un modelo de distribución de la riqueza dentro de la teoría de juegos. La **teoría de juegos** (o Game theory) trata de explicar las decisiones que toman unos individuos a fin de maximizar sus propios intereses dentro de una estructura formalizada de incentivos y penalizaciones, y estas estructuras son llamadas "juegos". En este caso, el modelo de valores de Shapley se aplica únicamente a los llamados "juegos cooperativos", es decir, aquellos en los que los participantes colaboran entre sí a fin de obtener una recompensa. 

El modelo funciona de la siguiente manera: la riqueza se redistribuirá entre los jugadores en función de la aportación que haya hecho cada uno de ellos para poder solucionar el problema. Así, el jugador que más haya aportado a llegar a la solución, se llevará una mayor parte de la recompensa total. 

Ahora bien, dicho modelo llevado al ámbito de machine learning implica ciertos cambios: el **juego** sería el propio modelo de machine learning en sí, los **"jugadores"** se identificarían con las **variables explicativas** y la **recompensa** sería la diferencia entre el modelo estimado con y sin esa variable, es decir, el valor adicional que aporta esa variable al modelo, premiando de esa manera el poder discriminatorio de cada variable. 

La función de reparto de la riqueza se define de la siguiente forma:

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <msub>
    <mi>&#x03D5;<!-- ϕ --></mi>
    <mi>j</mi>
  </msub>
  <mo stretchy="false">(</mo>
  <mi>v</mi>
  <mi>a</mi>
  <mi>l</mi>
  <mo stretchy="false">)</mo>
  <mo>=</mo>
  <munder>
    <mo>&#x2211;<!-- ∑ --></mo>
    <mrow class="MJX-TeXAtom-ORD">
      <mi>S</mi>
      <mo>&#x2286;<!-- ⊆ --></mo>
      <mo fence="false" stretchy="false">{</mo>
      <mn>1</mn>
      <mo>,</mo>
      <mo>&#x2026;<!-- … --></mo>
      <mo>,</mo>
      <mi>p</mi>
      <mo fence="false" stretchy="false">}</mo>
      <mo class="MJX-variant">&#x2216;<!-- ∖ --></mo>
      <mo fence="false" stretchy="false">{</mo>
      <mi>j</mi>
      <mo fence="false" stretchy="false">}</mo>
    </mrow>
  </munder>
  <mfrac>
    <mrow>
      <mrow class="MJX-TeXAtom-ORD">
        <mo stretchy="false">|</mo>
      </mrow>
      <mi>S</mi>
      <mrow class="MJX-TeXAtom-ORD">
        <mo stretchy="false">|</mo>
      </mrow>
      <mo>!</mo>
      <mrow>
        <mo>(</mo>
        <mrow>
          <mi>p</mi>
          <mo>&#x2212;<!-- − --></mo>
          <mrow class="MJX-TeXAtom-ORD">
            <mo stretchy="false">|</mo>
          </mrow>
          <mi>S</mi>
          <mrow class="MJX-TeXAtom-ORD">
            <mo stretchy="false">|</mo>
          </mrow>
          <mo>&#x2212;<!-- − --></mo>
          <mn>1</mn>
        </mrow>
        <mo>)</mo>
      </mrow>
      <mo>!</mo>
    </mrow>
    <mrow>
      <mi>p</mi>
      <mo>!</mo>
    </mrow>
  </mfrac>
  <mrow>
    <mo>(</mo>
    <mrow>
      <mi>v</mi>
      <mi>a</mi>
      <mi>l</mi>
      <mrow>
        <mo>(</mo>
        <mrow>
          <mi>S</mi>
          <mo>&#x222A;<!-- ∪ --></mo>
          <mo fence="false" stretchy="false">{</mo>
          <mi>j</mi>
          <mo fence="false" stretchy="false">}</mo>
        </mrow>
        <mo>)</mo>
      </mrow>
      <mo>&#x2212;<!-- − --></mo>
      <mi>v</mi>
      <mi>a</mi>
      <mi>l</mi>
      <mo stretchy="false">(</mo>
      <mi>S</mi>
      <mo stretchy="false">)</mo>
    </mrow>
    <mo>)</mo>
  </mrow>
</math>

* Donde ```ϕj val``` define el valor para la variable j en S. Suma y pondera todas las combinaciones posibles de valor de la variable.
* ```S```. Es el set de variables empleadas en el modelo. 
* ```p```. Es el número de variables del modelo
* ```S```
* ```S```

## Librerías

In [ ]:
# IMPORTAR LIBRERÍAS

## Scripts

In [ ]:
# SI LOS HUBIERA

## X.1 Capítulo 1

---

<div style='text-align:center'>Elaborado por Álvaro Serrano del Rincón (<i>a.serranodelrincon@cunef.edu</i>)</div> 
<div style='text-align:center'>y Carlos Viñals Guitart (<i>carlos.vinals@cunef.edu</i>)</div> 